# Bike Rental Analyses
## Introduction:
In a separate notebook i have prepared, cleansed and exported the data into a pgAdmin4 database. This document will show the code used to create the database. Afterward, i will showcase some of the SQL views used including screenshots of output and some data findings.

### Code to create the database, including key relationships:



```sql
CREATE TABLE bikes (
    Bike_ID bigint NOT NULL DEFAULT nextval('"bikes_Bike_ID_seq"'::regclass),
    PRIMARY KEY (Bike_ID)
);

CREATE TABLE stations (
    Station_ID bigint NOT NULL,
    Name text,
    Latitude double precision,
    Longitude double precision,
    PRIMARY KEY (Station_ID)
);

CREATE TABLE weather_stations (
    Weather_Station_ID text NOT NULL,
    Name text,
    PRIMARY KEY (Weather_Station_ID)
);

CREATE TABLE weather_data (
    Weather_Data_ID bigint NOT NULL DEFAULT nextval('"weather_data_Weather_Data_ID_seq"'::regclass),
    Weather_Station_ID text,
    Date timestamp without time zone,
    Precipitation_Inch double precision,
    Average_Wind_MPH double precision,
    Max_Temp_C double precision,
    Min_Temp_C double precision,
    Average_Temp_C double precision,
    Snowfall_Inch double precision,
    FOREIGN KEY (Weather_Station_ID) REFERENCES weather_stations(Weather_Station_ID)
);

CREATE TABLE users (
    User_ID bigint NOT NULL DEFAULT nextval('"users_User_ID_seq"'::regclass),
    Bike_ID bigint,
    User_Type text,
    Birth_Year double precision,
    Gender bigint,
    PRIMARY KEY (User_ID),
    FOREIGN KEY (Bike_ID) REFERENCES bikes(Bike_ID)
);

CREATE TABLE trips (
    Trip_ID integer NOT NULL DEFAULT nextval('"trips_Trip_ID_seq"'::regclass),
    Bike_ID bigint,
    Start_Station_ID bigint,
    End_Station_ID bigint,
    Start_Time timestamp without time zone,
    Stop_Time timestamp without time zone,
    Trip_Duration_Secs bigint,
    User_Type text,
    Birth_Year double precision,
    Gender bigint,
    FOREIGN KEY (Bike_ID) REFERENCES bikes(Bike_ID),
    FOREIGN KEY (Start_Station_ID) REFERENCES stations(Station_ID),
    FOREIGN KEY (End_Station_ID) REFERENCES stations(Station_ID)
);

## Below, we will showcase some views, outputs of these and findings.

SELECT 
    EXTRACT(MONTH FROM "Start_Time") as Month,
    COUNT("Trip_ID") as Trip_Count
FROM 
    trips
GROUP BY 
    EXTRACT(MONTH FROM "Start_Time")
ORDER BY 
    Month;

![Busiest Months](https://github.com/Max-Peacock/Bike-Rental/blob/main/image/Busiest%20Months.jpg?raw=true)



### This image shows the busiest months in the year, we can now inspect as to why these months are busier, perhaps there are some correlations we can look into

### The Below query will Extract the Month From Date, and change the ouput to the Months name, it will then group the months by the average rainful, and the average trip duration. This will give us information on how rainfall can affect a trips duration.
### Note: This is my first time using '::numeric', i had to find a way to convert the Datatype to ensure it can be in a more readable format (2 decimal places)
```sql
SELECT 
    CASE 
        WHEN EXTRACT(MONTH FROM "Date") = 1 THEN 'January'
        WHEN EXTRACT(MONTH FROM "Date") = 2 THEN 'February'
        WHEN EXTRACT(MONTH FROM "Date") = 3 THEN 'March'
        WHEN EXTRACT(MONTH FROM "Date") = 4 THEN 'April'
        WHEN EXTRACT(MONTH FROM "Date") = 5 THEN 'May'
        WHEN EXTRACT(MONTH FROM "Date") = 6 THEN 'June'
        WHEN EXTRACT(MONTH FROM "Date") = 7 THEN 'July'
        WHEN EXTRACT(MONTH FROM "Date") = 8 THEN 'August'
        WHEN EXTRACT(MONTH FROM "Date") = 9 THEN 'September'
        WHEN EXTRACT(MONTH FROM "Date") = 10 THEN 'October'
        WHEN EXTRACT(MONTH FROM "Date") = 11 THEN 'November'
        ELSE 'December'
    END as Month,
    ROUND(AVG("Precipitation_Inch")::numeric, 2) as Avg_Rainfall,
    ROUND(AVG("Trip_Duration_Secs"), 2) as Avg_Trip_Duration
FROM 
    weather_data 
JOIN 
    trips ON EXTRACT(MONTH FROM "Date") = EXTRACT(MONTH FROM "Start_Time")
GROUP BY 
    EXTRACT(MONTH FROM "Date")
ORDER BY
	Avg_Rainfall DESC;
Avg_Rainfall DESC;

<img src="C:/Users/maxpe/Jupyter Notebooks/Bike Rental/Avg Rain Trip Dur.jpg" alt="Avg Rain Trip Dur">

### Now we have the average rainfull for each month alongside the average trip duration
### As predicted, we can see below that busier months with longer trip durations are generally the drier months, with July being an outlier, lets inspect the weather and trip data in July and see the correlation between rainfall and trips.
### Note: we use '::date' after the Start_Time and Date on the JOIN, this will convert the current timestamp into a useable date format.
``` sql
SELECT 
    EXTRACT(DAY FROM wd."Date") as Day,
    ROUND(AVG(wd."Precipitation_Inch")::numeric, 2) as Avg_Rainfall,
    COUNT(t."Trip_ID") as No_Trips
FROM trips t
JOIN weather_data wd ON t."Start_Time"::date = wd."Date"::date
WHERE EXTRACT(MONTH FROM wd."Date") = 7
GROUP BY Day
ORDER BY No_Trips DESC;NTH FROM wd."Date");

<img src="C:/Users/maxpe/Jupyter Notebooks/Bike Rental/July Trips.jpg" alt="July Trips">

### We can see by looking at this query that although July had alot of rainfall and it was also busy, the busiest days of the month were also the days where no / low rainfall occured.

```sql
SELECT "Birth_Year", COUNT(*) as Birth_Year_Count
FROM users 
GROUP BY "Birth_Year"
ORDER BY Birth_Year_Count DESC
LIMIT 20;

<img src="C:/Users/maxpe/Jupyter Notebooks/Bike Rental/Birth Year.jpg" alt="Birth Year">

### We know from cleaning the data in pandas that the User_Type of 'Customer' does not have a corresponding Date of Birth and a Gender of 0 (unknown). However, we have Gender data from the User_Type 'Subscriber', we will run a query to see our Subscribers by Gender

``` sql
SELECT users."Gender", COUNT(*) as "User_Count"
FROM users
WHERE "User_Type" = 'Subscriber'
GROUP BY users."Gender"
ORDER BY "User_Count" DESC;

<img src="C:/Users/maxpe/Jupyter Notebooks/Bike Rental/Subscriber by Gender.jpg" alt="Subscriber by Gender">

### We can see the Gender of '1' has the most subscribers, this can now be used to help devise our target audience.
#### (we would need to check Data entry and see which Gender corresponds to '1' and '2', '0' is 'Unknown', we can also report that the User_Type of 'Customer' has no Birth Year)

### We will now check and see which Gender makes the longest trips on average.

```sql
SELECT
    users."Gender",
    AVG(trips."Trip_Duration_Secs"):: numeric (10,2) AS "AvgTripDuration"
FROM
    trips 
JOIN
    users ON trips."Bike_ID" = users."Bike_ID"
WHERE
    users."User_Type" = 'Subscriber'
GROUP BY
    users."Gender";."Gender";

<img src="C:/Users/maxpe/Jupyter Notebooks/Bike Rental/Trip Duration Gender.jpg" alt="Trip Duration Gender">

### As we can see, Gender 2 make the longest trip in general, but it is such a small difference, we can safely say that the average trip duration is not decided by gender!